In [4]:
#set workspace environment and variables

ws = r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\Desktop2025ProPro\Default.gdb"

profile = "Line Notes"  #feature respresenting the desired cross section(s)

projection = '''PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]];-110278200 -95394100 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision'''

arcpy.env.overwriteOutput = 1

#select the linear infrastructure (pipe, street, etc) to profile
#translate project to projection defined above next time
#arcpy.conversion.FeatureClassToFeatureClass("ssGravityMain", ws, profile)
#FeatureClassToFeatureClass is deprecated use 2025 - there goes 15 years of code :()
#each feature - street, sansewer, storm sewer can be profiled with data. Waterline profiles have to be made up.
#
xsfeature = "XS_W231"
with arcpy.EnvManager(outputCoordinateSystem=projection):
    arcpy.conversion.FeatureClassToFeatureClass(profile, ws, xsfeature)


#for streets 3d drapery on terrain will be used to set centerline profile slope
#explore sample distance, at 10 ft we will erase a lot of sumps and peaks
arcpy.ddd.InterpolateShape(
    in_surface=r"\\gisfile\gisdata\Published\LIDAR\2021_LiDAR\Lidar_Mosaic_Dataset.gdb\QL2_DEM_2021",
    in_feature_class=xsfeature,
    out_feature_class=ws+r"\\"+xsfeature+"3d",
    sample_distance=1,
    z_factor=1,
    method="BILINEAR",
    vertices_only="DENSIFY",
    pyramid_level_resolution=0,
    preserve_features="EXCLUDE"
)

#just match the terrain for 3d and create vertices
#random lines for XS should be digitized in the desired direction of display

arcpy.management.CalculateField(ws+r"/"+xsfeature+"3d", "StartPoint", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateGeometryAttributes(
    in_features=ws+r"//"+xsfeature+"3d",
    geometry_property="EndPoint LENGTH;EndZ LINE_END_Z;StartZ LINE_START_Z;Zmin EXTENT_MIN_Z;Zmax EXTENT_MAX_Z;BearingDirection LINE_BEARING;Length3D LENGTH_3D;MinX EXTENT_MIN_X;MinY EXTENT_MIN_Y",
    length_unit="FEET_US",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]]',
    coordinate_format="SAME_AS_INPUT"
)

#XS Lines - routes increase in line direction
arcpy.management.CalculateField(
    in_table="XSProfile3d",
    field="StartPt",
    expression="0",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="XSProfile3d",
    field="Name",
    expression="'Wisc231'",
    expression_type="PYTHON3",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table=ws+r"//"+xsfeature+"3d",
    field="Name",
    expression="'Wisc231'",
    expression_type="PYTHON3",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.lr.CreateRoutes(
    in_line_features=ws+r"//"+xsfeature+"3d",
    route_id_field="NAME",
    out_feature_class=ws+r"\XSRoute",
    measure_source="TWO_FIELDS",
    from_measure_field="StartPoint",
    to_measure_field="EndPoint",
    coordinate_priority="LOWER_LEFT",
    measure_factor=1,
    measure_offset=0,
    ignore_gaps="IGNORE",
    build_index="INDEX"
)
arcpy.management.FeatureVerticesToPoints(
    in_features=ws+r"//"+xsfeature+"3d",
    out_feature_class=ws+r"//"+xsfeature+"3dPt",
    point_location="ALL"
)
arcpy.management.CalculateGeometryAttributes(
    in_features=ws+r"//"+xsfeature+"3dPt",
    geometry_property="SpotZ POINT_Z",
    length_unit="",
    area_unit="",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

arcpy.lr.LocateFeaturesAlongRoutes(
    in_features=ws+r"//"+xsfeature+"3dPt",
    in_routes=ws+r"\XSRoute",
    route_id_field="Name",
    radius_or_tolerance="0 Feet",
    out_table=ws+r"//"+xsfeature+"3dPtM",
    out_event_properties="RID; Point; MEAS",
    route_locations="FIRST",
    distance_field="DISTANCE",
    zero_length_events="ZERO",
    in_fields="FIELDS",
    m_direction_offsetting="M_DIRECTON"
)    


#adjust the coordinates for the profile so it maps near the feature
#offset/plot math here is for primarily east and west streets

arcpy.management.CalculateField(
    in_table=ws+r"//"+xsfeature+"3dPtM",
    field="ProfileX",
    expression="!MEAS!+!MinX!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


arcpy.management.CalculateField(
    in_table=ws+r"//"+xsfeature+"3dPtM",
    field="ProfileY",
    expression="!SpotZ!+!MinY!-1000",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

    #version 3 create a fishnet grid, fishnet originates at upper right coordinate and nets to the left and down

#500 is the length
#840 is the top elev
#10 is the X spacing
#1 is the Y spacing
#15 is the Y num of rows
#50 is the X num of columns
#the coorindates below need to come from the data


Origin = "11498545.368264 556032.195484"
YAxisCoord = "11498545.368264 556132.195484"

arcpy.management.XYTableToPoint(
    in_table=ws+r"//"+xsfeature+"3dPtM",
    out_feature_class=ws+r"//"+xsfeature+"ProfilePt",
    x_field="ProfileX",
    y_field="ProfileY",
    z_field=None,
    coordinate_system='PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]];-110278200 -95394100 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision'
)

arcpy.management.PointsToLine(
    Input_Features=ws+r"//"+xsfeature+"ProfilePt",
    Output_Feature_Class=ws+r"//"+xsfeature+"ProfileLn",
    Line_Field="RID",
    Sort_Field="MEAS",
    Close_Line="NO_CLOSE",
    Line_Construction_Method="CONTINUOUS",
    Attribute_Source="NONE",
    Transfer_Fields=None)

<class 'SyntaxError'>: invalid syntax (C:/Users/KGONTE~1/AppData/Local/Temp/ArcGISProTemp9560/xpython_9560/3432207755.py, line 129)

In [ ]:
# Use create fishnet tool to create the fishnet with the extent of the profile line
#this is way easier than the coordinate processing to program the extent 
#to make sure the grid lines up with the profile basedd on the Y origin being located at "!SpotZ!+!MinY!-1000-mod(MinZ)"
# where Mod Min Z is the value of the remainder to the irght of hte decimal value


arcpy.management.CreateFishnet(ws+r"\\profilegrid", Origin, YAxisCoord, 10, 1, 25, 50, None, "LABELS", '0 0 0 0', "POLYLINE")


["'SpotZ!+!MinY!-1000' is not recognized as an internal or external command,", 'operable program or batch file.']


In [5]:
#labeling formatting for the grid
'''
arcpy.analysis.Buffer(
    in_features="profilegrid",
    out_feature_class=ws+r"\profilegrid_Buffer",
    buffer_distance_or_field="10 Feet",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)
'''
arcpy.management.CalculateField(
    in_table="profilegrid",
    field="ProfileElev",
    expression="!OID!+859",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.CalculateField(
    in_table="profilegrid",
    field="Station",
    expression="(!OID!-12)*10",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'profilegrid'>